### Finding the vol fitting outliers
   * produce the ATM option spread in 2021
   * find <20 periods of spikes
   * go to CT to check the vol fitting curves
   * write down the summaries into confluence

In [ ]:
from cdcqr.data.deribit.data_utils import quoto_and_trade_analysis
from datetime import datetime
import sys
import pandas as pd
from cdcqr.data.deribit.data_utils import get_spread_from_ATM_option_quote_data
from cdcqr.common.utils import parallel_jobs
from cdcqr.common.config import LOCAL_DATA_DIR
sys.path.append('C://github/cryptoderiv-quant/')

import matplotlib.pyplot as plt

In [ ]:
from ct.utils import plot2

quoto_and_trade_analysis?

In [ ]:
ret_dict, ret_df = quoto_and_trade_analysis(contract='BTC-25JUN21-60000-C', date='2021-04-18')

In [ ]:
ret_df.head()

In [ ]:
ret_df.set_index('timestamp_dt')['spread'].pipe(df2)

### 1) Load quote data, finding the ATM option, produce the spread at minutes level

In [ ]:
def get_spread_from_ATM_option_quote_data(date):
    """
    load quote data -> find the ATM call -> calcuate spread, resample at 1 min level -> return the max spread time series
    """
    df = data_loader(exchange='deribit', date = date.strftime('%Y-%m-%d'), data_type='quotes', symbol='OPTIONS', )
    df1 = df[['symbol','timestamp','ask_price','bid_price']]
    df2 = df1[df1['symbol'].str.contains('BTC') & df1['symbol'].str.contains('-C')]
    df2['spread'] = df2['ask_price'] - df2['bid_price']
    df3 = df2[['symbol','timestamp','spread']]
    ATM_call = df3.groupby('symbol')['spread'].count().sort_values(ascending=False).index[0]
    df4 = df3.query('symbol==@ATM_call')
    df4['dt'] = pd.to_datetime(df4['timestamp'], unit='us')
    df5 = df4.set_index('dt').resample('1T')['spread'].max()
    df5 = df5[df5.index>=date]
    return df5

In [ ]:
from cdcqr.data.dataloader import data_loader
dates = pd.date_range(datetime(2021,1,1), datetime(2021,10,28))
date = dates[0]

In [ ]:
get_spread_from_ATM_option_quote_data(date)

### 2) run the above function in parallel

In [ ]:
dates = pd.date_range(datetime(2021,1,1), datetime(2021,4,1))
dates2 = pd.date_range(datetime(2021,4,2), datetime(2021,7,1))
dates3 = pd.date_range(datetime(2021,7,2), datetime(2021,10,27))


In [ ]:
ret_dict = parallel_jobs(get_spread_from_ATM_option_quote_data, dates3)

In [ ]:
ret_df = pd.concat(ret_dict.values()).sort_index()

In [ ]:
ret_df_08_10= ret_df.copy()

In [ ]:
ret_df_04_07 = ret_df.copy()

In [ ]:
ret_df.plot()

In [ ]:
ret_df_01_03 = ret_df.copy()

In [ ]:
ret_df_04_07.plot()

In [ ]:
df_2021[(df_2021['dt']>=datetime(2021,7,8,0,0)) & (df_2021['dt']<datetime(2021,7,9,0,0))].set_index('dt')['spread'].plot()

In [ ]:
df_2021[(df_2021['dt']>=datetime(2021,5,23,0,0)) & (df_2021['dt']<datetime(2021,5,24,0,0))].set_index('dt')['spread'].plot()

In [ ]:
ret_df_04_07.shape

In [ ]:
ret_df_08_10.shape

In [ ]:
df_2021 = ret_df_01_03.append(ret_df_04_07).append(ret_df_08_10)

In [ ]:
df_2021['date'] = df_2021['dt'].dt.date

In [ ]:
df_2021.to_pickle()

### 3) find the spike periods

In [ ]:
df_2021.plot()

In [ ]:
df_2021 = df_2021.reset_index()

#### spikes >=0.5

In [ ]:
spikes_lv1 = df_2021[df_2021>=0.5].reset_index()

In [ ]:
spikes_lv1['date'] = spikes_lv1['dt'].dt.date

In [ ]:
spikes_lv1.groupby('date').count()

In [ ]:
df_2021[(df_2021['dt']>=datetime(2021,9,27,0,0)) & (df_2021['dt']<datetime(2021,9,28,0,0))].set_index('dt')['spread'].plot()

In [ ]:
### check ATM bid/ask quantity, 

In [ ]:
ret_dict, ret_df_40C = quoto_and_trade_analysis(contract='BTC-31DEC21-40000-C', date='2021-05-19')

In [ ]:
ret_dict, ret_df_40P = quoto_and_trade_analysis(contract='BTC-31DEC21-40000-P', date='2021-05-19')

In [ ]:
ret_dict, ret_df_32P = quoto_and_trade_analysis(contract='BTC-31DEC21-32000-P', date='2021-05-19')

In [ ]:
ret_dict, ret_df_32C = quoto_and_trade_analysis(contract='BTC-31DEC21-32000-C', date='2021-05-19')

In [ ]:
ret_dict, ret_df_48C = quoto_and_trade_analysis(contract='BTC-31DEC21-48000-C', date='2021-05-19')

In [ ]:
ret_dict, ret_df_48P = quoto_and_trade_analysis(contract='BTC-31DEC21-48000-P', date='2021-05-19')

In [ ]:
df2plot = ret_df_40C.query('timestamp_dt>=datetime(2021,5,19,22,30)').query('timestamp_dt<=datetime(2021,5,19,23,30)').set_index('timestamp_dt')[['ask_price','bid_price','flow']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_price','bid_price',]].plot(ax=ax1, linestyle='None', marker = '.')
ax2 = ax1.twinx()
df2plot['flow'].plot(ax=ax2, legend=['trade'], color = ['b'])
plt.title('BTC-31DEC21-40000-C')
plt.show()

In [ ]:
df2plot = ret_df_40P.query('timestamp_dt>=datetime(2021,5,19,22,30,15)').query('timestamp_dt<=datetime(2021,5,19,22,30,20)').set_index('timestamp_dt')[['ask_price','bid_price','flow']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_price','bid_price',]].plot(ax=ax1, linestyle='None', marker = '.')
ax2 = ax1.twinx()
df2plot['flow'].plot(ax=ax2, legend=['trade'], color = ['b'])
plt.title('BTC-31DEC21-40000-P')
plt.show()

In [ ]:
df2plot = ret_df_40P.query('timestamp_dt>=datetime(2021,5,19,22,30,15)').query('timestamp_dt<=datetime(2021,5,19,22,30,20)').set_index('timestamp_dt')[['ask_amount','bid_amount']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_amount',]].plot(ax=ax1, linestyle='None', marker = '.',  legend=['bid_amount'])
plt.title('BTC-31DEC21-40000-P')
plt.show()

In [ ]:
ret_df_40P.query('timestamp_dt>=datetime(2021,5,19,22,30,15)').query('timestamp_dt<=datetime(2021,5,19,22,30,20)')[['timestamp_dt','','','','']]

In [ ]:
df2plot = ret_df_32P.query('timestamp_dt>=datetime(2021,5,19,22,30)').query('timestamp_dt<=datetime(2021,5,19,23,30)').set_index('timestamp_dt')[['ask_price','bid_price','flow']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_price','bid_price',]].plot(ax=ax1, linestyle='None', marker = '.')
ax2 = ax1.twinx()
df2plot['flow'].plot(ax=ax2, legend=['trade'], color = ['b'])
plt.title('BTC-31DEC21-32000-P')
plt.show()

In [ ]:
df2plot = ret_df_32C.query('timestamp_dt>=datetime(2021,5,19,22,30)').query('timestamp_dt<=datetime(2021,5,19,23,30)').set_index('timestamp_dt')[['ask_price','bid_price','flow']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_price','bid_price',]].plot(ax=ax1, linestyle='None', marker = '.')
ax2 = ax1.twinx()
df2plot['flow'].plot(ax=ax2, legend=['trade'], color = ['b'])
plt.title('BTC-31DEC21-32000-C')
plt.show()

In [ ]:
#df2plot = ret_df_48P.query('timestamp_dt>=datetime(2021,5,19,22,30)').query('timestamp_dt<=datetime(2021,5,19,23,30)').set_index('timestamp_dt')[['ask_price','bid_price','flow']]
df2plot = ret_df_48P.query('timestamp_dt>=datetime(2021,5,19,15,30)').query('timestamp_dt<=datetime(2021,5,19,15,35)').set_index('timestamp_dt')[['ask_price','bid_price','flow']]

fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_price','bid_price',]].plot(ax=ax1, linestyle='None', marker = '.')
ax2 = ax1.twinx()
df2plot['flow'].plot(ax=ax2, legend=['trade'], color = ['b'], linestyle='None', marker = 'x')
plt.title('BTC-31DEC21-48000-P')
plt.show()

In [ ]:
df2plot = ret_df_48P.query('timestamp_dt>=datetime(2021,5,19,22,30)').query('timestamp_dt<=datetime(2021,5,19,23,30)').set_index('timestamp_dt')[['ask_price','bid_price','flow']]
df2plot = ret_df_48P.set_index('timestamp_dt')[['ask_price','bid_price','flow']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_price','bid_price',]].plot(ax=ax1, linestyle='None', marker = '.')
ax2 = ax1.twinx()
df2plot['flow'].plot(ax=ax2, legend=['trade'], color = ['b'])
plt.title('BTC-31DEC21-48000-P')
plt.show()

In [ ]:
df = data_loader(exchange='deribit', date = '2021-05-19', data_type='quotes', symbol='OPTIONS', )

In [ ]:
[x for x in list(df['symbol'].unique()) if 'BTC-31DEC21' in x]

In [ ]:
quoto_and_trade_analysis??

In [ ]:
exchange = 'deribit'
date = '2021-05-19'
symbol = 'OPTIONS'
df_opt_trade = data_loader(exchange, date,"trades",symbol)

In [ ]:
df_opt_trade = df_opt_trade.query('symbol=="BTC-31DEC21-40000-P"')

In [ ]:
df_opt_trade['dt'] = pd.to_datetime(df_opt_trade['timestamp'], unit='us') # check convert ts

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
df_opt_trade.set_index('dt')

In [ ]:
ret_df_48P.query('timestamp_dt>=datetime(2021,5,19,15,00)').query('timestamp_dt<=datetime(2021,5,19,16,00)')